In [1]:
import logging
import requests
import pandas as pd

In [2]:
url_list = []
for year in range(2009,2019):
    concat_url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(concat_url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    url_list.extend(game_ids)

In [3]:
def parse_data(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if item[0] == "visitorTeam":
                    continue
                elif item[0] == "homeTeam":
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif item[0] == "gameId":
                    game_id = item[-1]
                elif item[0] == "site":
                    for it in item[-1].items():
                        row.append(it[-1])
                else:
                    row.append(item[-1])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    tot = item[-1].get('pointTotal')
                    row.append(tot)
    stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[-1])
    return row


def get_columns(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if item[0] == "visitorTeam":
                    continue
                elif item[0] == "homeTeam":
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif item[0] == "gameId":
                    game_id = item[-1]
                elif item[0] == "site":
                    for it in item[-1].items():
                        row.append(it[0])
                else:
                    row.append(item[0])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    row.append(item[0])
    stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[0])
    return row


def main(urls):
    """ Runs data processing scripts to turn raw data from (../raw) into
        cleaned data ready to be analyzed (saved in ../processed).
    """
    data = []
    counter = 0
    for l in urls[1:]:
        try:
            row = parse_data(l)
            data.append(row)
            counter += 1
            print(counter, "out of", len(url_list), "completed...")
        except:
            print("Skipped...")

    url_for_cols = urls[0]
    all_columns = get_columns(url_for_cols)
    game_columns = all_columns[:24]
    stat_columns = all_columns[24:]
    cols = stat_columns[:109]

    home_columns = []
    for col in cols:
        home_columns.append("H{}".format(col))

    visitor_columns = []
    for col in cols:
        visitor_columns.append("V{}".format(col))

    final_columns = []
    final_columns.extend(game_columns)
    final_columns.extend(home_columns)
    final_columns.extend(visitor_columns)

    df = pd.DataFrame(data)
    df.columns = final_columns
    df.to_csv("../data/interim/game_data.csv", index=False)


    return df

In [4]:
df = main(url_list)

1 out of 3328 completed...
2 out of 3328 completed...
3 out of 3328 completed...
4 out of 3328 completed...
5 out of 3328 completed...
6 out of 3328 completed...
7 out of 3328 completed...
8 out of 3328 completed...
9 out of 3328 completed...
10 out of 3328 completed...
11 out of 3328 completed...
12 out of 3328 completed...
13 out of 3328 completed...
14 out of 3328 completed...
15 out of 3328 completed...
16 out of 3328 completed...
17 out of 3328 completed...
18 out of 3328 completed...
19 out of 3328 completed...
20 out of 3328 completed...
21 out of 3328 completed...
22 out of 3328 completed...
23 out of 3328 completed...
24 out of 3328 completed...
25 out of 3328 completed...
26 out of 3328 completed...
27 out of 3328 completed...
28 out of 3328 completed...
29 out of 3328 completed...
30 out of 3328 completed...
31 out of 3328 completed...
32 out of 3328 completed...
33 out of 3328 completed...
34 out of 3328 completed...
35 out of 3328 completed...
36 out of 3328 completed...
3

288 out of 3328 completed...
289 out of 3328 completed...
290 out of 3328 completed...
291 out of 3328 completed...
292 out of 3328 completed...
293 out of 3328 completed...
294 out of 3328 completed...
295 out of 3328 completed...
296 out of 3328 completed...
297 out of 3328 completed...
298 out of 3328 completed...
299 out of 3328 completed...
300 out of 3328 completed...
301 out of 3328 completed...
302 out of 3328 completed...
303 out of 3328 completed...
304 out of 3328 completed...
305 out of 3328 completed...
306 out of 3328 completed...
307 out of 3328 completed...
308 out of 3328 completed...
309 out of 3328 completed...
310 out of 3328 completed...
311 out of 3328 completed...
312 out of 3328 completed...
313 out of 3328 completed...
314 out of 3328 completed...
315 out of 3328 completed...
316 out of 3328 completed...
317 out of 3328 completed...
318 out of 3328 completed...
319 out of 3328 completed...
320 out of 3328 completed...
321 out of 3328 completed...
322 out of 332

571 out of 3328 completed...
572 out of 3328 completed...
573 out of 3328 completed...
574 out of 3328 completed...
575 out of 3328 completed...
576 out of 3328 completed...
577 out of 3328 completed...
578 out of 3328 completed...
579 out of 3328 completed...
580 out of 3328 completed...
581 out of 3328 completed...
582 out of 3328 completed...
583 out of 3328 completed...
584 out of 3328 completed...
585 out of 3328 completed...
586 out of 3328 completed...
587 out of 3328 completed...
588 out of 3328 completed...
589 out of 3328 completed...
590 out of 3328 completed...
591 out of 3328 completed...
592 out of 3328 completed...
593 out of 3328 completed...
594 out of 3328 completed...
595 out of 3328 completed...
596 out of 3328 completed...
597 out of 3328 completed...
598 out of 3328 completed...
599 out of 3328 completed...
600 out of 3328 completed...
601 out of 3328 completed...
602 out of 3328 completed...
603 out of 3328 completed...
604 out of 3328 completed...
605 out of 332

854 out of 3328 completed...
855 out of 3328 completed...
856 out of 3328 completed...
857 out of 3328 completed...
858 out of 3328 completed...
859 out of 3328 completed...
860 out of 3328 completed...
861 out of 3328 completed...
862 out of 3328 completed...
863 out of 3328 completed...
864 out of 3328 completed...
865 out of 3328 completed...
866 out of 3328 completed...
867 out of 3328 completed...
868 out of 3328 completed...
869 out of 3328 completed...
870 out of 3328 completed...
871 out of 3328 completed...
872 out of 3328 completed...
873 out of 3328 completed...
874 out of 3328 completed...
875 out of 3328 completed...
876 out of 3328 completed...
877 out of 3328 completed...
878 out of 3328 completed...
879 out of 3328 completed...
880 out of 3328 completed...
881 out of 3328 completed...
882 out of 3328 completed...
883 out of 3328 completed...
884 out of 3328 completed...
885 out of 3328 completed...
886 out of 3328 completed...
887 out of 3328 completed...
888 out of 332

1132 out of 3328 completed...
1133 out of 3328 completed...
1134 out of 3328 completed...
1135 out of 3328 completed...
1136 out of 3328 completed...
1137 out of 3328 completed...
1138 out of 3328 completed...
1139 out of 3328 completed...
1140 out of 3328 completed...
1141 out of 3328 completed...
1142 out of 3328 completed...
1143 out of 3328 completed...
1144 out of 3328 completed...
1145 out of 3328 completed...
1146 out of 3328 completed...
1147 out of 3328 completed...
1148 out of 3328 completed...
1149 out of 3328 completed...
1150 out of 3328 completed...
1151 out of 3328 completed...
1152 out of 3328 completed...
1153 out of 3328 completed...
1154 out of 3328 completed...
1155 out of 3328 completed...
1156 out of 3328 completed...
1157 out of 3328 completed...
1158 out of 3328 completed...
1159 out of 3328 completed...
1160 out of 3328 completed...
1161 out of 3328 completed...
1162 out of 3328 completed...
1163 out of 3328 completed...
1164 out of 3328 completed...
1165 out o

...

2228 out of 3328 completed...
2229 out of 3328 completed...
2230 out of 3328 completed...
2231 out of 3328 completed...
2232 out of 3328 completed...
2233 out of 3328 completed...
2234 out of 3328 completed...
2235 out of 3328 completed...
2236 out of 3328 completed...
2237 out of 3328 completed...
2238 out of 3328 completed...
2239 out of 3328 completed...
2240 out of 3328 completed...
2241 out of 3328 completed...
2242 out of 3328 completed...
2243 out of 3328 completed...
2244 out of 3328 completed...
2245 out of 3328 completed...
2246 out of 3328 completed...
2247 out of 3328 completed...
2248 out of 3328 completed...
2249 out of 3328 completed...
2250 out of 3328 completed...
2251 out of 3328 completed...
2252 out of 3328 completed...
2253 out of 3328 completed...
2254 out of 3328 completed...
2255 out of 3328 completed...
2256 out of 3328 completed...
2257 out of 3328 completed...
2258 out of 3328 completed...
2259 out of 3328 completed...
2260 out of 3328 completed...
2261 out o

2501 out of 3328 completed...
2502 out of 3328 completed...
2503 out of 3328 completed...
2504 out of 3328 completed...
2505 out of 3328 completed...
2506 out of 3328 completed...
2507 out of 3328 completed...
2508 out of 3328 completed...
2509 out of 3328 completed...
2510 out of 3328 completed...
2511 out of 3328 completed...
2512 out of 3328 completed...
2513 out of 3328 completed...
2514 out of 3328 completed...
2515 out of 3328 completed...
2516 out of 3328 completed...
2517 out of 3328 completed...
2518 out of 3328 completed...
2519 out of 3328 completed...
2520 out of 3328 completed...
2521 out of 3328 completed...
2522 out of 3328 completed...
2523 out of 3328 completed...
2524 out of 3328 completed...
2525 out of 3328 completed...
2526 out of 3328 completed...
2527 out of 3328 completed...
2528 out of 3328 completed...
2529 out of 3328 completed...
2530 out of 3328 completed...
2531 out of 3328 completed...
2532 out of 3328 completed...
2533 out of 3328 completed...
2534 out o

2774 out of 3328 completed...
2775 out of 3328 completed...
2776 out of 3328 completed...
2777 out of 3328 completed...
2778 out of 3328 completed...
2779 out of 3328 completed...
2780 out of 3328 completed...
2781 out of 3328 completed...
2782 out of 3328 completed...
2783 out of 3328 completed...
2784 out of 3328 completed...
2785 out of 3328 completed...
2786 out of 3328 completed...
2787 out of 3328 completed...
2788 out of 3328 completed...
2789 out of 3328 completed...
2790 out of 3328 completed...
2791 out of 3328 completed...
2792 out of 3328 completed...
2793 out of 3328 completed...
2794 out of 3328 completed...
2795 out of 3328 completed...
2796 out of 3328 completed...
2797 out of 3328 completed...
2798 out of 3328 completed...
2799 out of 3328 completed...
2800 out of 3328 completed...
2801 out of 3328 completed...
2802 out of 3328 completed...
2803 out of 3328 completed...
2804 out of 3328 completed...
2805 out of 3328 completed...
2806 out of 3328 completed...
2807 out o

3047 out of 3328 completed...
3048 out of 3328 completed...
3049 out of 3328 completed...
3050 out of 3328 completed...
3051 out of 3328 completed...
3052 out of 3328 completed...
3053 out of 3328 completed...
3054 out of 3328 completed...
3055 out of 3328 completed...
3056 out of 3328 completed...
3057 out of 3328 completed...
3058 out of 3328 completed...
3059 out of 3328 completed...
3060 out of 3328 completed...
3061 out of 3328 completed...
3062 out of 3328 completed...
3063 out of 3328 completed...
3064 out of 3328 completed...
3065 out of 3328 completed...
3066 out of 3328 completed...
3067 out of 3328 completed...
3068 out of 3328 completed...
3069 out of 3328 completed...
3070 out of 3328 completed...
3071 out of 3328 completed...
3072 out of 3328 completed...
3073 out of 3328 completed...
3074 out of 3328 completed...
3075 out of 3328 completed...
3076 out of 3328 completed...
3077 out of 3328 completed...
3078 out of 3328 completed...
3079 out of 3328 completed...
3080 out o

3321 out of 3328 completed...
3322 out of 3328 completed...
Skipped...
3323 out of 3328 completed...
